# Function Generation for the Training of λ-Nets

## Specitication of Experiment Settings

In [39]:
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utilities.DecisionTree import SoftBinaryDecisionTree, brand_new_tfsession, draw_tree, draw_tree_image
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, Callback
from sklearn.preprocessing import OneHotEncoder
import os
import glob

import numpy as np
import torch
np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

In [3]:
X, y = load_breast_cancer(return_X_y=True)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

enc = OneHotEncoder()
y = enc.fit_transform(y.reshape(-1, 1)).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [23]:
X, y = load_iris(return_X_y=True)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

enc = OneHotEncoder()
y = enc.fit_transform(y.reshape(-1, 1)).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [24]:
X_train.shape

(100, 4)

In [25]:
y_test.shape

(50, 3)

In [46]:
#n_features = img_rows * img_cols * img_chans
tree_depth = 1
penalty_strength = 0#1e+1
penalty_decay = 0#0.25
ema_win_size = 1000
inv_temp = 0#0.01
learning_rate = 0.01#5e-03
batch_size = 1#4

In [27]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
%autoreload 2
sess = brand_new_tfsession()

tree = SoftBinaryDecisionTree(tree_depth, 
                              n_features=X_train.shape[1], 
                              n_classes=y_train.shape[1],
                              penalty_strength=penalty_strength, 
                              penalty_decay=penalty_decay,
                              inv_temp=inv_temp, 
                              ema_win_size=ema_win_size, 
                              learning_rate=learning_rate)

tree.build_model()

/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Built tree has 2 leaves out of 3 nodes


In [29]:
X_train.shape

(100, 4)

In [30]:
y_train.shape

(100, 3)

In [47]:
epochs = 40

es = EarlyStopping(monitor='val_acc', patience=20, verbose=1)

'''If you wish to train your own model instead of loading one from checkpoint, remove the checkpoint.'''
os.remove('assets/non-distilled/checkpoint')
for f in glob.glob('assets/non-distilled/tree-model*'):
    os.remove(f)

tree.maybe_train(
    sess=sess, data_train=(X_train, y_train), data_valid=(X_test, y_test),
    batch_size=batch_size, epochs=epochs, callbacks=[es])

Loading trained model from assets/non-distilled/tree-model.
assets/non-distilled/tree-model is not a valid checkpoint. Training from scratch.
Train on 100 samples, validate on 50 samples
Epoch 1/40
100/100 [==============================] - 0s 2ms/sample - loss: 0.0959 - acc: 0.3500 - val_loss: 0.0954 - val_acc: 0.3000
Epoch 2/40
100/100 [==============================] - 0s 2ms/sample - loss: 0.0950 - acc: 0.3500 - val_loss: 0.0955 - val_acc: 0.3000
Epoch 3/40
100/100 [==============================] - 0s 2ms/sample - loss: 0.0947 - acc: 0.3500 - val_loss: 0.0956 - val_acc: 0.3000
Epoch 4/40
100/100 [==============================] - 0s 2ms/sample - loss: 0.0942 - acc: 0.3500 - val_loss: 0.0957 - val_acc: 0.3000
Epoch 5/40
100/100 [==============================] - 0s 2ms/sample - loss: 0.0936 - acc: 0.3500 - val_loss: 0.0954 - val_acc: 0.3200
Epoch 6/40
100/100 [==============================] - 0s 2ms/sample - loss: 0.0929 - acc: 0.3500 - val_loss: 0.0953 - val_acc: 0.3200
Epoch 7/4

In [48]:
kernels = dict([(l.name.split('_')[-1],
                 np.squeeze(l.get_weights()[0]))
                for l in tree.model.layers if 'dense' in l.name])
biases = dict([(l.name.split('_')[-1], np.squeeze(l.get_weights()[1][0]))
               for l in tree.model.layers if 'dense' in l.name])
leaves = dict([(l.name.split('_')[-1], sess.run(l.output))
               for l in tree.model.layers if 'pdist' in l.name])

In [49]:
kernels

{'0': array([-0.57247144,  1.7576524 , -2.406543  , -2.512611  ], dtype=float32)}

In [50]:
biases

{'0': 1.0149852}

In [51]:
leaves

{'00': array([[0.87572837, 0.06968661, 0.05458508]], dtype=float32),
 '01': array([[0.09978398, 0.41757396, 0.48264208]], dtype=float32)}

In [ ]:
tree= SDT(input_dim=X_train.shape[1], 
          output_dim=int(max(y_train))+1, 
          depth=1,
          use_cuda=False)

In [ ]:
tree

In [ ]:
tree.fit(X_train, y_train, batch_size=32, epochs=10)

In [ ]:
tree.evaluate(X_test, y_test)

In [ ]:
index = 1

m1 = torch.FloatTensor(X_test[index:index+1])
print(m1.shape)
print(m1)

m1_new = torch.reshape(torch.cat((torch.FloatTensor([1]), m1[0])), (1, torch.cat((torch.FloatTensor([1]), m1[0])).shape[0]))
print(m1_new.shape)
print(m1_new)

m2=tree.inner_nodes[0].weight
print(m2.shape)
print(m2)

m3 = tree.leaf_nodes.weight
print(m3.shape)
print(m3)

In [ ]:
adjust_index = 5

m2_custom = torch.FloatTensor([[0 for i in range(31)]])
m2_custom[0][1+adjust_index] = 1
print(m2_custom.shape)
print(m2_custom)

m2 = m2_custom

print(m1[0][adjust_index])

In [ ]:
adjust_index = 5

m1_custom = torch.FloatTensor([[0 for i in range(31)]])
m1_custom[0][0] = 0#1
m1_custom[0][1+adjust_index] = -3
print(m1_custom.shape)
print(m1_custom)

m1_new = m1_custom


print(m1[0][adjust_index])

In [ ]:
m3 = torch.FloatTensor([[1,0], [1,0]]) #niedrige werte in erstem array (stehen für klasse 0) stehen für geringe wahrscheinlichkeit dieser klasse

In [ ]:
m2

In [ ]:
torch.matmul(m2[0], m1_new[0])

In [ ]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [ ]:
sigmoid(3)

In [ ]:
0.3168 * -1.6799 + 0.6832 * 1.0056

In [ ]:
0.3168 * 1.1978 + 0.6832 * -0.6925

In [ ]:
#output = torch.stack([torch.stack([torch.sigmoid(torch.matmul(m1[0], m2[0][1:])+m2[0][0])])])
print(m2[0]) # input
print(m1_new[0]) # filter
output = torch.stack([torch.stack([torch.sigmoid(torch.matmul(m2[0], m1_new[0]))])])
print(output)
output = torch.unsqueeze(output, dim=2)
print(output)
output = torch.cat((output, 1 - output), dim=2)
print(output)
print(m3) #output distrib
output = torch.stack([torch.matmul(m3, output[0][0])])
print(output)
output = torch.softmax(output, dim=1)
print(output)
output = output.data.max(1)[1]
print(output)

In [ ]:
%load_ext autoreload

In [ ]:
import copy
tree_copy = copy.deepcopy(tree)

In [ ]:
adjust_index = 5

m2_custom = torch.FloatTensor([[0 for i in range(31)]])
m2_custom[0][1+adjust_index] = -2
print(m2_custom.shape)
print(m2_custom)

m2 = m2_custom

print(m1[0][adjust_index])

In [ ]:
tree_copy.inner_nodes[0].weight = torch.nn.Parameter(m2_custom)
print(tree_copy.inner_nodes[0].weight)

In [ ]:
tree_copy.leaf_nodes.weight = torch.nn.Parameter(torch.FloatTensor([[-1,1], [1,-1]]))

In [ ]:
sigmoid(0.2562*2)

In [ ]:
%autoreload 2
index = 1
tree_copy.predict(X_test[index:index+1])

In [ ]:
np.array([-0.4566, 0.3367])

In [ ]:
t1 = torch.FloatTensor([[-0.4566, 0.3367]])

In [ ]:
torch.softmax(t1, dim=1)

In [ ]:
print(tree.inner_nodes[0].weight.shape)
print(tree.inner_nodes[0].weight)

In [ ]:
print(tree.leaf_nodes.weight.shape)
print(tree.leaf_nodes.weight)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

normal_tree = DecisionTreeClassifier(max_depth=1)
normal_tree.fit(X_train, y_train)

In [ ]:
preds = normal_tree.predict(X_test)
print(accuracy_score(y_test, preds))

In [ ]:
plt.figure(figsize=(20,12))
plot_tree(normal_tree, fontsize=12)

plt.show()

In [ ]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'function_family': {
        'maximum_depth': 3,
        'fully_grown': True,       
        'balance': 0.5
        'balancing_tolerance': 0.05               
    }
    'data': {
        'number_of_variables': 3, 
        'num_classes': 2,
        
        'function_generation_type': 'make_classification', #'random'
        'objective': 'classification' # 'multiclass_classification', 'regression'
        
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform',       
        
        'same_training_all_lambda_nets': False,
        
        'lambda_dataset_size': 5000, #number of samples per polynomial
        'number_of_generated_datasets': 10000,
    }, 
    'computation':{
        'n_jobs': 5,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 0,   
    }
}


In [ ]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['computation'])

## Imports

In [ ]:
from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 

import os
import sys

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler

from sympy import Symbol, sympify

        
import seaborn as sns
        
import random 

import warnings

from time import perf_counter

In [ ]:
#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################
sns.set_style("darkgrid")

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')

In [ ]:
from utilities.utility_functions import *
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################

    
#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['computation'])

initialize_utility_functions_config_from_curent_notebook(config)

#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(path_type='data_creation'))
generate_directory_structure()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [ ]:
print(path_identifier_polynomial_data)

# Function Generation

In [ ]:
def generate_random_decision_tree():
    
    
    
number_of_variables,
maximum_depth,
num_classes,
fully_grown
    

In [ ]:
def generate_decision_tree_data(n_samples, noise, noise_dist, seed):
    
    decision_tree = generate_random_decision_tree()
    
    return decision_tree, X_data, y_data

In [ ]:
if same_training_all_lambda_nets:
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='multiprocessing')
    result_list = parallel(delayed(generate_decision_tree_data)(polynomial_array=list_of_polynomials[i], 
                                                               n_samples=lambda_dataset_size,
                                                               noise=noise,
                                                               noise_dist=noise_distrib, 
                                                               seed=RANDOM_SEED, 
                                                               sympy_calculation=False) for i in range(polynomial_data_size))  
else:
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='multiprocessing')
    result_list = parallel(delayed(gen_regression_symbolic)(polynomial_array=list_of_polynomials[i], 
                                                               n_samples=lambda_dataset_size,
                                                               noise=noise,
                                                               noise_dist=noise_distrib, 
                                                               seed=RANDOM_SEED+i, 
                                                               sympy_calculation=False) for i in range(polynomial_data_size))

X_data_list = [[pd.Series(result[0],  index=list_of_monomial_identifiers_string), pd.DataFrame(result[1], columns=list(variables[:n]))] for result in result_list]
y_data_list = [[pd.Series(result[0],  index=list_of_monomial_identifiers_string), pd.DataFrame(result[2], columns=['result'])] for result in result_list]



In [ ]:
X_data_list[0][0].head(10)

In [ ]:
X_data_list[0][1].head(10)

In [ ]:
y_data_list[0][0].head(10)

In [ ]:
y_data_list[0][1].head(10)

In [ ]:
path_polynomials = './data/saved_polynomial_lists/polynomials_sample_' + path_identifier_polynomial_data + '.csv'
polynomials_list_df.to_csv(path_polynomials, index=False)

path_X_data = './data/saved_polynomial_lists/X_sample_' + path_identifier_polynomial_data + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(X_data_list, f)#, protocol=2)
    
path_y_data = './data/saved_polynomial_lists/y_sample_' + path_identifier_polynomial_data + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(y_data_list, f)#, protocol=2)
